# TMBD Project - Assignment 1 - Bayesian Network

### [Bank costumer churn](https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset)


 Developed by:  
 Beatriz Gonçalves – 115367​  
 Ana Valente – 93307​  
 Obafemi Raymondjoy – 111949

## Table of contents

1. [Required libraries](#Required-libraries)

2. [Data Cleaning](#DataCleaning)

3. [Bayesian Netwok](#INE_main_agricultural_crops-DataCleaning-OneIndex)

    3.1 .[Discrete Distribution](#Discrete-distribution)  
    3.2 .[Conditional Probabilities](#Conditional-probabilities)  
    3.3 .[Network Representation](#Conditional-probabilities)  
    3.4 .[Example](#Example)    
    
4. [References](#References)

## Required libraries

[[ go back to the top ]](#Table-of-contents)

If you don't have Python on your computer, you can use the [Anaconda Python distribution](http://continuum.io/downloads) to install most of the Python packages you need. Anaconda provides a simple double-click installer for your convenience.

This notebook uses some Python packages that come standard with the Anaconda Python distribution. The primary libraries that we'll be using are:
* **pandas**: Provides a DataFrame structure to store data in memory and work with it easily and efficiently.
* **pomegranate**: Implements fast and flexible probabilistic models ranging from individual probability distributions to compositional models such as Bayesian networks and hidden Markov models.
* **pygraphviz**: Graph layout and visualization package.
* **matplotlib**: Comprehensive library for creating static, animated, and interactive visualizations.

In [1]:
#from google.colab import files
#files.upload()
!pip install pomegranate 

!pip install matplotlib
!pip install graphviz

!apt install libgraphviz-dev
!pip install pygraphviz


import pandas as pd
from pomegranate import *
import pygraphviz as pgv
import matplotlib.pyplot as plt
%pylab inline

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
  Using cached pygraphviz-1.7.zip (118 kB)
  ERROR: Command errored out with exit status 1:
   command: /home/beatriz/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-jvlcb6dh/pygraphviz/setup.py'"'"'; __file__='"'"'/tmp/pip-install-jvlcb6dh/pygraphviz/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-iw175_i2
       cwd: /tmp/pip-install-jvlcb6dh/pygraphviz/
  Complete output (53 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/pygraphviz
  copying pygraphviz/scraper.py -> build/lib.linux-x86_64-3.7/pygra

ModuleNotFoundError: No module named 'pygraphviz'

## Data Cleaning

[[ go back to the top ]](#Table-of-contents)

In [3]:
# Loading Data

data = pd.read_csv("Bank Customer Churn Prediction.csv") 

In [4]:
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


We can get some information about our dataset, as for the number of entries and columns.

In [5]:
data.info() # 10000 observations and 12 variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


Check if there are any missing values.

In [6]:
data.isnull().sum()
# There are no missing values

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

Check if there's data duplicated.

In [7]:
data[data.duplicated()]
# There's no duplicated rows 

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn


Satistics on our dataset variables.

In [8]:
data.describe()

,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Remove the columns we think may not be important to our analysis.

In [9]:
data = data.drop(['customer_id', 'products_number'], axis=1)

Variables such as credit_score, age, tenure, balance and estimated_salary have a lot of unique values. In order to compute the probabilities of each we set bands in them.

In [10]:
# create bands to show in the dataframe
data['credit_score'] = data['credit_score'].apply(lambda x: '350-449' if (x >= 350 and x < 450) else
                                                  '450-549' if (x >= 450 and x < 550) else
                                                  '550-649' if (x >= 550 and x < 650) else
                                                  '650-749' if (x >= 650 and x < 750) else
                                                  '750-850')

data['age'] = data['age'].apply(lambda x: '15-34' if (x >= 15 and x < 35) else 
                                '35-54' if (x >= 35 and x < 55) else
                                '55-74' if (x >= 55 and x < 75) else
                                '75-95')

data['tenure'] = data['tenure'].apply(lambda x: '0-4' if (x >= 0 and x <= 4) else '5-10')

data['balance'] = data['balance'].apply(lambda x: '0-50k' if (x >= 0 and x < 50000) else
                                  '50k-100k' if (x >= 50000 and x < 100000) else
                                  '100k-150k' if (x >= 100000 and x < 150000) else
                                  '150k-200k' if (x >= 150000 and x < 200000) else 
                                  '200k-255k')

data['estimated_salary'] = data['estimated_salary'].apply(lambda x: '0-50k' if (x >= 0 and x < 50000) else
                                  '50k-100k' if (x >= 50000 and x < 100000) else
                                  '100k-150k' if (x >= 100000 and x < 150000) else
                                  '150k-200k')

In [11]:
data

,credit_score,country,gender,age,tenure,balance,credit_card,active_member,estimated_salary,churn
0,550-649,France,Female,35-54,0-4,0-50k,1,1,100k-150k,1
1,550-649,Spain,Female,35-54,0-4,50k-100k,0,1,100k-150k,0
2,450-549,France,Female,35-54,5-10,150k-200k,1,0,100k-150k,1
3,650-749,France,Female,35-54,0-4,0-50k,0,0,50k-100k,0
4,750-850,Spain,Female,35-54,0-4,100k-150k,1,1,50k-100k,0
...,...,...,...,...,...,...,...,...,...,...
9995,750-850,France,Male,35-54,5-10,0-50k,1,0,50k-100k,0
9996,450-549,France,Male,35-54,5-10,50k-100k,1,1,100k-150k,0
9997,650-749,France,Female,35-54,5-10,0-50k,0,1,0-50k,1
9998,750-850,Germany,Male,35-54,0-4,50k-100k,1,0,50k-100k,1


## Bayesian Network

[[ go back to the top ]](#Table-of-contents)

### Discrete Distribution

[[ go back to the top ]](#Table-of-contents)

Create distributions for the independent events, such as country, age and gender.

In [ ]:
country = DiscreteDistribution({'France': data['country'].value_counts(normalize = True)[0], 
                                'Germany': data['country'].value_counts(normalize = True)[1],
                                'Spain': data['country'].value_counts(normalize = True)[2]})

age = DiscreteDistribution({'15-34': data['age'].value_counts(normalize = True)[1],
                            '35-54': data['age'].value_counts(normalize = True)[0],
                            '55-74': data['age'].value_counts(normalize = True)[2],
                            '75-95': data['age'].value_counts(normalize = True)[3]})

gender = DiscreteDistribution({'Male': data['gender'].value_counts(normalize = True)[0],
                               'Female': data['gender'].value_counts(normalize = True)[1]})

### Conditional Probabilities

[[ go back to the top ]](#Table-of-contents)

**Estimated salary** may depend on **country from residence** and **gender**.

In [ ]:
psalary = data.groupby(['country', 'gender'])['estimated_salary'].value_counts(normalize = True).reset_index(name = 'probability')

In [ ]:
psalary.values.tolist()

In [ ]:
estimated_salary = ConditionalProbabilityTable(psalary.values.tolist(), [country, gender])

In [ ]:
estimated_salary

**Tenure** concerns to the amount of years a client has an account on the bank. That may be correlated with the **age**.

In [ ]:
# tenure - from how many years a client has an account on the bank
ptenure = data.groupby(['age'])['tenure'].value_counts(normalize = True).reset_index(name = 'probability')

In [ ]:
ptenure.values.tolist()

In [ ]:
tenure = ConditionalProbabilityTable(ptenure.values.tolist(), [age])

The **account balance** may be influenced by the **tenure** and **age**.

In [ ]:
pbalance = data.groupby(['age', 'tenure'])['balance'].value_counts(normalize = True).reset_index(name = 'probability')

In [ ]:
row1 = pd.DataFrame({'age': '75-95', 'tenure': '0-4', 
                     'balance':  '200k-255k', 'probability': 0}, index = [35])
row2 = pd.DataFrame({'age': '75-95', 'tenure': '5-10', 
                     'balance':  '200k-255k', 'probability': 0}, index = [38])
df = pd.concat([pbalance.iloc[:34], row1, pbalance.iloc[34:]]).reset_index(drop = True)
bal = df.append(row2, ignore_index = True)

In [ ]:
# Since there are conditional probabilities which hold 0, those have to be insert manually
rows = pd.DataFrame({'age': '75-95', 'tenure': ['0-4','5-10'], 'balance': '200k-255k', 'probability': 0})
bal = pbalance.append(rows, ignore_index = True)

In [ ]:
bal.values.tolist()

In [ ]:
balance = ConditionalProbabilityTable(bal.values.tolist(), [age, tenure])

In [ ]:
balance

The **credit score** may depend on **balance** and **estimated_salary**.

In [ ]:
pcredit = data.groupby(['balance', 'estimated_salary'])['credit_score'].value_counts(normalize = True).reset_index(name = 'probability')

In [ ]:
pcredit.values.tolist()

In [ ]:
# Some of the conditional probabilities are 0, in those cases they have to be inserted manually
row1 = pd.DataFrame({'balance': '200k-255k', 'estimated_salary': ['50k-100k', '150k-200k', '100k-150k', '0-50k'],
                    'credit_score': '350-449', 'probability': 0})
row2 = pd.DataFrame({'balance': '200k-255k', 'estimated_salary': '100k-150k',
                    'credit_score': '750-850', 'probability': 0}, index = [-1])
pcredit1 = pcredit.append(row1, ignore_index = True)
cred = pcredit1.append(row2, ignore_index = True)

In [ ]:
cred.values.tolist()

In [ ]:
credit_score = ConditionalProbabilityTable(cred.values.tolist(), [balance, estimated_salary])

In [ ]:
credit_score

Having a **credit card** may depend on **age**. 

In [ ]:
pcreditcard = data.groupby(['age'])['credit_card'].value_counts(normalize = True).reset_index(name = 'probability')

In [ ]:
credit_card = ConditionalProbabilityTable(pcreditcard.values.tolist(), [age])

In [ ]:
credit_card

Being an **active member** may be correlated with having (or not) a **credit card** and with the **credit_score**.

In [ ]:
pactive = data.groupby(['credit_card', 'credit_score'])['active_member'].value_counts(normalize = True).reset_index(name = 'probability')

In [ ]:
active_member = ConditionalProbabilityTable(pactive.values.tolist(), [credit_card, credit_score])

In [ ]:
active_member

In [ ]:
pchurn = data.groupby(['active_member'])['churn'].value_counts(normalize = True).reset_index(name = 'probability')

In [ ]:
churn = ConditionalProbabilityTable(pchurn.values.tolist(), [active_member])

In [ ]:
pchurn
# Since it's not explicit in the information on dataset, we consider 0 - being an active member, 1 - not being an active member

In [ ]:
churn

### Network Representation

[[ go back to the top ]](#Table-of-contents)

Create the states of the Bayesian Network.

In [ ]:
s1 = State(gender, name = 'gender')
s2 = State(country, name = 'country')
s3 = State(age, name = 'age')
s4 = State(estimated_salary, name = 'estimated_salary')
s5 = State(tenure, name = 'tenure')
s6 = State(credit_card, name = 'credit_card')
s7 = State(balance, name = 'balance')
s8 = State(credit_score, name = 'credit_score')
s9 = State(active_member, name = 'active_member')
s10 = State(churn, name = 'churn')

In [ ]:
network = BayesianNetwork("Bank customer churn")
network.add_nodes(s1, s2, s3, s4, s5, s6, s7, s8, s9, s10)

In [ ]:
network.add_transition(s1,s4)
network.add_transition(s2,s4)
network.add_transition(s3,s5)
network.add_transition(s3,s6)
network.add_transition(s4,s8)
network.add_transition(s5,s7)
network.add_transition(s6,s9)
network.add_transition(s7,s8)
network.add_transition(s8,s9)
network.add_transition(s9,s10)

In [ ]:
network.bake()

In [ ]:
print("\t".join([state.name for state in network.states]))

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7, 7)
network.plot()

### Example
[[ go back to the top ]](#Table-of-contents)

In [12]:
# needs to be done

## References

[[ go back to the top ]](#Table-of-contents)

 Info about the pomegranate library: https://pomegranate.readthedocs.io/en/latest/  
 Info about the pygraphviz library: https://pygraphviz.github.io/  
 Info about the matplotlib library: https://matplotlib.org/stable/index.html  
 